In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/TrueCue/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TrueCue


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle

## Import latest Covid data from Our World in Data

In [3]:
df=pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv') 
df.head().T

,0,1,2,3,4
iso_code,ABW,ABW,ABW,ABW,ABW
continent,North America,North America,North America,North America,North America
location,Aruba,Aruba,Aruba,Aruba,Aruba
date,2020-03-13,2020-03-19,2020-03-20,2020-03-21,2020-03-22
total_cases,2,NaN,4,NaN,NaN
new_cases,2,NaN,2,NaN,NaN
new_cases_smoothed,NaN,0.286,0.286,0.286,0.286
total_deaths,NaN,NaN,NaN,NaN,NaN
new_deaths,0,NaN,0,NaN,NaN
new_deaths_smoothed,NaN,0,0,0,0


## Extract new_cases: new confirmed cases of COVID-19

In [4]:
df_pvt=df.pivot(index='date', columns='iso_code', values='new_cases').fillna(0.)
df_pvt.tail()

iso_code,NaN,ABW,AFG,AGO,AIA,ALB,AND,ARE,ARG,ARM,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BWA,CAF,CAN,CHE,CHL,CHN,CIV,CMR,...,SMR,SOM,SRB,SSD,STP,SUR,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TLS,TTO,TUN,TUR,TWN,TZA,UGA,UKR,URY,USA,UZB,VAT,VCT,VEN,VGB,VIR,VNM,WLF,YEM,ZAF,ZMB,ZWE
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-11-02,0.0,6.0,132.0,247.0,0.0,327.0,69.0,1278.0,6609.0,1194.0,0.0,5.0,4526.0,1175.0,4.0,2660.0,0.0,0.0,0.0,1568.0,1225.0,278.0,0.0,1415.0,977.0,0.0,0.0,76.0,10100.0,0.0,0.0,5.0,0.0,0.0,2330.0,0.0,1608.0,24.0,0.0,0.0,...,0.0,0.0,1449.0,21.0,4.0,7.0,2282.0,1342.0,1303.0,8.0,5.0,0.0,61.0,0.0,15.0,26.0,3.0,37.0,0.0,0.0,791.0,2106.0,5.0,0.0,248.0,7959.0,25.0,81001.0,101.0,0.0,1.0,312.0,0.0,0.0,0.0,0.0,0.0,1371.0,48.0,12.0
2020-11-03,0.0,5.0,95.0,230.0,0.0,321.0,63.0,1234.0,9598.0,1328.0,0.0,7.0,3160.0,596.0,0.0,13345.0,40.0,0.0,17.0,1736.0,2427.0,210.0,21.0,764.0,941.0,162.0,7.0,34.0,8501.0,1.0,0.0,2.0,0.0,0.0,3422.0,21842.0,1324.0,49.0,37.0,0.0,...,0.0,0.0,802.0,0.0,0.0,2.0,1883.0,557.0,1573.0,4.0,13.0,2.0,54.0,0.0,1.0,7.0,10.0,42.0,0.0,36.0,0.0,2302.0,4.0,0.0,228.0,6754.0,16.0,83883.0,98.0,0.0,0.0,380.0,0.0,2.0,12.0,0.0,0.0,772.0,63.0,15.0
2020-11-04,0.0,14.0,86.0,193.0,0.0,381.0,22.0,1008.0,12145.0,2374.0,2.0,8.0,4445.0,1242.0,8.0,11167.0,0.0,0.0,13.0,1659.0,4041.0,230.0,55.0,1553.0,929.0,166.0,0.0,69.0,11843.0,0.0,0.0,2.0,0.0,0.0,4672.0,6090.0,1014.0,16.0,12.0,310.0,...,66.0,288.0,1878.0,14.0,5.0,6.0,1727.0,1177.0,3627.0,13.0,15.0,2.0,45.0,0.0,14.0,17.0,7.0,43.0,0.0,50.0,1220.0,2343.0,1.0,0.0,128.0,8899.0,31.0,92734.0,299.0,0.0,0.0,395.0,0.0,7.0,11.0,0.0,4.0,1241.0,118.0,21.0
2020-11-05,0.0,15.0,121.0,349.0,0.0,396.0,135.0,1161.0,10652.0,2413.0,0.0,12.0,6514.0,1227.0,2.0,0.0,62.0,0.0,9.0,1517.0,4054.0,261.0,53.0,1776.0,984.0,115.0,0.0,126.0,23976.0,1.0,0.0,0.0,0.0,9.0,2768.0,10043.0,840.0,27.0,13.0,0.0,...,0.0,0.0,2412.0,3.0,4.0,2.0,3216.0,2025.0,3213.0,13.0,11.0,0.0,76.0,1.0,4.0,25.0,6.0,41.0,0.0,10.0,1237.0,2391.0,1.0,0.0,252.0,9524.0,49.0,102507.0,226.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,1712.0,37.0,17.0
2020-11-06,0.0,11.0,40.0,236.0,0.0,421.0,90.0,1289.0,11100.0,2210.0,0.0,11.0,7051.0,1364.0,1.0,0.0,0.0,1.0,11.0,1842.0,3754.0,162.0,39.0,1908.0,982.0,72.0,1.0,139.0,0.0,0.0,0.0,0.0,1193.0,0.0,3635.0,10098.0,1540.0,32.0,11.0,0.0,...,49.0,0.0,2181.0,2.0,0.0,5.0,1962.0,1686.0,0.0,12.0,7.0,1.0,76.0,0.0,12.0,36.0,8.0,39.0,0.0,10.0,1971.0,2311.0,4.0,0.0,217.0,9850.0,64.0,122436.0,360.0,0.0,1.0,821.0,0.0,2.0,0.0,0.0,0.0,1866.0,72.0,17.0


## Make a dataframe of the 14 day notification rate per 100_000 population

In [5]:
df_14_day=pd.DataFrame()
for col in df_pvt.columns:
  df_14_day[col]=df_pvt[col].rolling(window=14).sum()
df_14_day.fillna(0., inplace=True)

for col in df_14_day.columns[1:]:
  popn=(df.loc[(df.date=='2020-10-01') & (df.iso_code==col), 'population']/100000).values
  if len(popn) > 0: # avoid dividing by 0
    df_14_day[col]=df_14_day[col]/popn
  else: 
    print (col)

df_14_day = df_14_day.T
df_14_day.head().T

MHL
SLB
WLF


,NaN,ABW,AFG,AGO,AIA
date,,,,,
2019-12-31,0.0,0.000000,0.000000,0.000000,0.0
2020-01-01,0.0,0.000000,0.000000,0.000000,0.0
2020-01-02,0.0,0.000000,0.000000,0.000000,0.0
2020-01-03,0.0,0.000000,0.000000,0.000000,0.0
2020-01-04,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...
2020-11-02,0.0,184.515670,3.681123,10.171523,0.0
2020-11-03,0.0,177.959275,3.701673,10.384507,0.0
2020-11-04,0.0,171.402881,3.696536,10.341910,0.0


In [6]:
df_14_day.loc[['ESP','GBR']].T.tail(7) # Spain no data on weekends, delivered too late for Fridays

,ESP,GBR
date,,
2020-10-31,532.818215,442.636158
2020-11-01,532.818215,451.097401
2020-11-02,569.456177,460.337892
2020-11-03,579.713951,460.554432
2020-11-04,596.972079,458.621780
2020-11-05,598.944070,456.397463
2020-11-06,556.486381,460.669330


## Colour scales, country codes and cartogram cells

In [7]:
cut_labels=['<20','20-59.9','60-119.9','120-239.9','240-479.9','480-959.9','>960']
cut_bins = [-1, 20., 60., 120., 240., 480., 960., 2000.]
color_dict={'<20':'lemonchiffon','20-59.9':'yellow','60-119.9':'orange','120-239.9':'sandybrown','240-479.9':'brown','480-959.9':'red', '>960':'black'}

iso_3166=pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/slim-3/slim-3.csv',
                     dtype={'name':'str','alpha-3':'str','country-code':'float64'})

cells = pd.read_csv("https://raw.githubusercontent.com/mattdzugan/World-Population-Cartogram/master/data/year_2018__cell_500k/squares_and_triangles/cells.csv")

sq1 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.7, markersize=8, markerfacecolor="lemonchiffon")
sq2 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.7, markersize=8, markerfacecolor="yellow")
sq3 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.7, markersize=8, markerfacecolor="orange")
sq4 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.7, markersize=8, markerfacecolor="sandybrown")
sq5 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.7, markersize=8, markerfacecolor="brown")
sq6 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.7, markersize=8, markerfacecolor="red")
sq7 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.7, markersize=8, markerfacecolor="black")

## Plot for each day

In [20]:
for date in df_14_day.columns[[-1*(1+i*1) for i in range((pd.to_datetime(df_14_day.columns[-1])-pd.to_datetime('2020-04-30')).days)][::-1]]:
#for date in [df_14_day.columns[-1]]:
  one_day_df = pd.DataFrame(df_14_day[date])
  one_day_df['count'] = pd.cut(one_day_df[date], bins=cut_bins, labels=cut_labels)
  one_day_df['color'] = one_day_df['count'].map(color_dict)
  one_day_df = one_day_df.reset_index().rename(columns={'index':'CountryCode'})
  one_day_df = one_day_df.merge(iso_3166, left_on='CountryCode', right_on='alpha-3', how='left')
  one_day_df['country-code'] = one_day_df['country-code'].fillna(0).astype('int')

  cells_day = cells.merge(one_day_df[['country-code','color']], left_on='CountryCode', right_on='country-code', how='left')
  cells_day.color.fillna('lemonchiffon', inplace=True)

  fig = plt.figure(figsize=(16,9))
  ax = fig.add_subplot(111, aspect='equal')
  plt.xlim([0, max(cells_day["X"]+1)])
  plt.ylim([0, max(cells_day["Y"]+1)])
  patches = []

  for i in range(cells_day.shape[0]):
    patches.append(Rectangle((cells_day.loc[i,"X"]+.5, cells_day.loc[i,"Y"]+.5),1,1))
  ax.add_collection(PatchCollection(patches, color=cells_day.color, alpha=0.7))

  plt.legend((sq1, sq2, sq3, sq4, sq5, sq6, sq7), (cut_labels), numpoints=1, loc=(.0,.77), title='1 cell: 500 000 people\n\nRate', fontsize=10, frameon=False)
  title =  '14-day COVID-19 case notification rate per 100 000 population\nCartogram for ' + date
  plt.title(title, fontsize=14)
  plt.axis('off')
  plt.figtext(0.5, 0.12, "Data sources: 'Our World in Data' and https://github.com/mattdzugan/World-Population-Cartogram\nCode: https://github.com/AlisonDavey ", ha='center', fontsize=12)
  plt.tight_layout()
  plt.savefig('./world_cartogram/pngs_world/'+date, dpi=150)
  #plt.show()
  plt.close()

# Write contents of the 'pngs' directory to a .gif
(very slow)

In [21]:
!pip install pygifsicle 
!apt install gifsicle

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gifsicle is already the newest version (1.91-2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [22]:
import os
import imageio
from pygifsicle import optimize

In [23]:
png_dir = './world_cartogram/pngs_world/'
images = []

for file_name in sorted(os.listdir(png_dir)):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))

imageio.mimwrite('./world_cartogram/world.gif', images, format='.gif', fps=10, loop=1)
optimize('./world_cartogram/world.gif') 